# Convolutional Neural Networks

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

In [9]:
device = torch.device('cuda')

<br>

We will be using the CIFAR-10 data set which is a multiclass image classification dataset with 10 classes.

The dataset will have ```PILImage``` images of range 0 to 1. We'll transform the tensors to range -1 to 1.

In [10]:
transform = transforms.Compose([
                transforms.ToTensor(), # converts PIL to tensor and also scales the pixel values
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # normalizes RGB channel values z = x-mean/std
            ])


train_dataset = torchvision.datasets.CIFAR10(
                    root='./data', 
                    train=True,
                    download=True,
                    transform = transform
                )

test_dataset = train_dataset = torchvision.datasets.CIFAR10(
                    root='./data', 
                    train=False,
                    transform = transform
                )

batch_size = 8

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle=False)

Files already downloaded and verified


In [19]:
samples = iter(train_loader).next()

samples

[tensor([[[[ 0.0667,  0.0980,  0.1137,  ...,  0.5373,  0.4902,  0.4588],
           [ 0.0510,  0.0824,  0.0980,  ...,  0.4431,  0.4039,  0.3725],
           [-0.0118,  0.0196,  0.0431,  ...,  0.3255,  0.3176,  0.3098],
           ...,
           [-0.3725, -0.3647, -0.3647,  ..., -0.4353, -0.4353, -0.4588],
           [-0.3569, -0.3569, -0.3569,  ..., -0.4353, -0.4588, -0.4745],
           [-0.3725, -0.3804, -0.3725,  ..., -0.4275, -0.4431, -0.4902]],
 
          [[ 0.1765,  0.2078,  0.2235,  ...,  0.5922,  0.5373,  0.5216],
           [ 0.1608,  0.1922,  0.2157,  ...,  0.5059,  0.4588,  0.4431],
           [ 0.0980,  0.1294,  0.1529,  ...,  0.3961,  0.3804,  0.3961],
           ...,
           [-0.4353, -0.4275, -0.4275,  ..., -0.4745, -0.4745, -0.4902],
           [-0.4196, -0.4196, -0.4196,  ..., -0.4745, -0.4980, -0.5059],
           [-0.4353, -0.4431, -0.4275,  ..., -0.4667, -0.4824, -0.5216]],
 
          [[ 0.5451,  0.5686,  0.5843,  ...,  0.8588,  0.8275,  0.8118],
           [ 

In [11]:
# class labels
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [22]:
# build our CNN
class ConvNet(nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        
        # convolutional and pooling layers
        self.conv1 = nn.Conv2d(
                        in_channels = 3,  # number of input channels
                        out_channels = 6,  # number of channels outputted; equivalent to "filters" from Keras
                        kernel_size= 5,  # size of the convolutional kernel
                        device='cuda'
                    )
        
        self.pool = nn.MaxPool2d(2,2) # kernel_size and stride
        
        self.conv2 = nn.Conv2d(
                        in_channels = 6,
                        out_channels = 16,
                        kernel_size = 5,
                        device='cuda'
                    )
        
        # hidden layers
        self.hidden1 = nn.Linear(16*5*5, 120, device='cuda')
        self.hidden2 = nn.Linear(120, 84, device='cuda')
        self.output = nn.Linear(84, 10, device='cuda')
        
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = self.output(x)
        
        return x

In [35]:
cnn = ConvNet()

learning_rate = 0.001

# define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(cnn.parameters(), lr = learning_rate, momentum=0.9)



In [36]:
num_epochs = 5

# define training loop
for epoch in range(num_epochs):
    
    running_loss = 0.0
    
    for i, data in enumerate(train_loader,0):
        inputs, labels = data
        # we have to send the input and labels to the GPU
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        # feed forward
        outputs = cnn(inputs)
        # compute loss
        loss = criterion(outputs, labels)
        # backprop
        loss.backward()
        # descend
        optimizer.step()
        
        # print loss computed as the average loss over every 100 batches
        running_loss += loss.item()
        if (i+1)%100 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, batch {i+1}/{len(train_loader)}, loss = {running_loss*0.01}')
            running_loss = 0.0
            
            
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = cnn(inputs)
        
        _, predictions = torch.max(outputs, 1)
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()
        
    acc = 100*n_correct/n_samples
    
    print(f'val accuracy: {acc}')

epoch 1/5, batch 100/1250, loss = 2.304778971672058
epoch 1/5, batch 200/1250, loss = 2.304095549583435
epoch 1/5, batch 300/1250, loss = 2.305702602863312
epoch 1/5, batch 400/1250, loss = 2.300153121948242
epoch 1/5, batch 500/1250, loss = 2.299937663078308
epoch 1/5, batch 600/1250, loss = 2.298501777648926
epoch 1/5, batch 700/1250, loss = 2.29862229347229
epoch 1/5, batch 800/1250, loss = 2.295178837776184
epoch 1/5, batch 900/1250, loss = 2.2920383548736574
epoch 1/5, batch 1000/1250, loss = 2.2877686476707457
epoch 1/5, batch 1100/1250, loss = 2.273166217803955
epoch 1/5, batch 1200/1250, loss = 2.2448321557044983
epoch 2/5, batch 100/1250, loss = 2.1740983891487122
epoch 2/5, batch 200/1250, loss = 2.1467120957374575
epoch 2/5, batch 300/1250, loss = 2.0654223418235778
epoch 2/5, batch 400/1250, loss = 2.11178759098053
epoch 2/5, batch 500/1250, loss = 2.0577013051509856
epoch 2/5, batch 600/1250, loss = 2.017982339859009
epoch 2/5, batch 700/1250, loss = 2.0065340876579283
epo